In [1]:
file = 'data/directed/ego-twitter/train.in'
N = 10

In [2]:
import tools

In [3]:
train_set, nodes, poss_set, neg_set = tools.sample_structural(file, N, directed=True)

In [4]:
from graph_tool.all import *
g = Graph(directed=False)
g.add_vertex(2*max(nodes) + 1)

<generator object <genexpr> at 0x7fc08d89feb0>

In [5]:
print 2*max(nodes) + 1

46741


In [ ]:
for edge in train_set:
    if edge not in poss_set:
        u, w = map(int, edge.split())
        g.add_edge(g.vertex(2*u - 1), g.vertex(2*w))

In [ ]:
pos = sfdp_layout(g, max_iter=4)

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
features = tools.TopologicalFeatures(g, pos, directed=True)
X, Y = tools.make_dataset(poss_set, neg_set, 
                    [features.dist])
roc_auc_score(Y, X)

In [ ]:
from graph_tool.all import *
g = Graph(directed=False)
g.add_vertex(max(nodes) + 1)

for edge in train_set:
    if edge not in poss_set:
        u, w = map(int, edge.split())
        g.add_edge(g.vertex(u), g.vertex(w))


features = tools.TopologicalFeatures(g)
X, Y = tools.make_dataset(poss_set, neg_set, 
                    [features.preferential_attachment])
roc_auc_score(Y, X)

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
from sklearn.decomposition import NMF
model = NMF(n_components=10, init='random', random_state=0)

matrix = tools.make_sparse_matrix(train_set, nodes, poss_set, directed=True)

features = tools.MFFeatures(model, matrix)
X, Y = tools.make_dataset(poss_set, neg_set, 
                    [features.score])
roc_auc_score(Y, X)

In [ ]:
from scipy.sparse import linalg
import numpy

matrix = tools.make_sparse_matrix(train_set, nodes, poss_set, directed=True)
U, s, Vh = linalg.svds(matrix.asfptype(), k=30)

def score(u, w):
    return numpy.dot(U[u] * s, Vh.T[w])

X, Y = tools.make_dataset(poss_set, neg_set, 
                    [score])
roc_auc_score(Y, X)